In [ ]:
!nvidia-smi

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/kaggle.json ~/.kaggle/

In [ ]:
!pip install --upgrade kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d koryakinp/chess-positions

In [ ]:
!unzip chess-positions.zip -d /content/Datas

In [ ]:

import re
import cv2
import glob
import random as rd
import warnings
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from imutils import paths
%matplotlib inline

In [ ]:


train_path = glob.glob(r"/content/Datas/train/*.jpeg")
test_path = glob.glob(r"/content/Datas/test/*.jpeg")

rd.shuffle(train_path)
rd.shuffle(test_path)

train_size = 8000
test_size = 2000
train = train_path[:train_size]
test= test_path[:test_size]
print(len(train))
print(len(test))

piece_symbols = 'prbnkqPRBNKQ'


In [ ]:
def get_image_FEN_label(image_path):
    fen_label= image_path.replace('.jpeg', '').split('/')[-1]
    fen_label = fen_label.replace('train\\', '')
    return fen_label

In [ ]:
rand = np.random.randint(0, train_size)
img_path = train[rand]
print(img_path)
img_moves = get_image_FEN_label(img_path)
print(img_moves)
img_rand = cv2.imread(img_path)
plt.imshow(cv2.cvtColor(img_rand, cv2.COLOR_BGR2RGB))
plt.title(img_moves)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
!pip install chess

In [ ]:
import chess.svg
import chess
print("The FEN notation of the image is: ", img_moves)
board = chess.Board(img_moves.replace('-', '/'))
chess.svg.board(board, size=300)

In [ ]:
samples =rd.sample(train, 9)
fig = plt.figure(figsize=(11, 11))
columns = 3
rows = 3
for i, img in zip(range(1, columns*rows +1),samples ):
    fig.add_subplot(rows, columns, i)
    img_moves =  get_image_FEN_label(img)
    img = cv2.imread(img)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(img_moves)
    plt.tight_layout()

plt.show()

In [ ]:
ratios = []
widths = []
heights = []

for img in train:
  img= cv2.imread(img)
  ratios.append(img.shape[1] / img.shape[0])
  heights.append(img.shape[0])
  widths.append(img.shape[1])

fig, (ax1,ax2,ax3) = plt.subplots(1, 3, figsize=(12, 5))

ax1.hist(ratios, bins=50)
ax1.set_xlabel('ratio')
ax1.set_ylabel('count')
ax1.set_title('Average ratio: {}'.format(np.mean(ratios)))

ax2.hist(widths, bins=50)
ax2.set_xlabel('width')
ax2.set_ylabel('count')
ax2.set_title('Average width: {}'.format(np.mean(widths)))

ax3.hist(heights, bins=50)
ax3.set_xlabel('height')
ax3.set_ylabel('count')
ax3.set_title('Average height: {}'.format(np.mean(heights)))

print("Selected Width X heights: {}X{}".format(int(np.mean(widths)) ,int(np.mean(heights))))

In [ ]:
def preprocess_some_images(img_paths, width, height):
  resized_imgs = []
  for img_path in img_paths:
    img = cv2.imread(img_path, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.resize(img, (width, height))
    gray_image =(gray_image - np.min(gray_image)) / (np.max(gray_image) - np.min(gray_image))
    resized_imgs.append(gray_image)
  return resized_imgs
preprocessed_imgs= preprocess_some_images(samples,240,240)


In [ ]:
"{}X{}".format(preprocessed_imgs[0].shape[0],preprocessed_imgs[0].shape[1])


In [ ]:
preprocessed_imgs[0].shape


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=15)
img_transformed = pca.fit_transform(new_img)

percentage_var_explained = pca.explained_variance_ / np.sum(pca.explained_variance_)
cum_var_explained = np.cumsum(percentage_var_explained)

reserved = np.round(np.sum(pca.explained_variance_ratio_), 3) * 100
print("Using {} components reserves {}% of the features".format(pca.n_components, reserved))

x_vals = np.arange(pca.n_components + 1)
y_vals = np.insert(cum_var_explained, 0, 0)

plt.figure(figsize=(6, 4))
plt.plot(x_vals, y_vals, linewidth=2)
plt.grid(True)
plt.xlabel('n_components')
plt.ylabel('Cumulative_explained_variance')

plt.xlim(0, pca.n_components)
plt.ylim(0, 1.0)

plt.axhline(y=reserved / 100.0, linestyle='--', color='k', linewidth=2)
plt.tight_layout()
plt.show()


In [ ]:
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(12, 5))
img = np.array(cv2.imread(samples[8]))
new_img=img.reshape(img.shape[0], (img.shape[1]*img.shape[2]))
new_img= cv2.resize(new_img, (720, 720),interpolation=cv2.INTER_CUBIC)
ax1.imshow(new_img, cmap='gray')
ax1.set_title("Before PCA image")

plt.figure(figsize=(6, 6))
temp = pca.inverse_transform(img_transformed)
temp = np.reshape(temp, (720,720))
ax2.imshow(temp, cmap='gray')
ax2.set_title("After PCA image")

In [ ]:
pip install visualkeras

In [ ]:
def image_to_squares_pca(img,heights,widths):
  squares = []
  for i in range(0,8):
    for j in range(0,8):
      new_img =img[i*heights//8:i*heights//8+heights//8,j*widths//8:j*widths//8+widths//8]
      new_img=new_img.reshape(new_img.shape[0], (new_img.shape[1]*new_img.shape[2]))
      new_img= cv2.resize(new_img, (720, 720),interpolation=cv2.INTER_CUBIC)
      squares.append(new_img)
  return np.array(squares)

img = np.array(cv2.imread(samples[8]))
sqaures= image_to_squares_pca(img,200,200)
fig = plt.figure(figsize=(10, 10))
columns = 8
rows = 8
for i, img in zip(range(1, columns*rows +1),sqaures):
    fig.add_subplot(rows, columns, i)
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.tight_layout()
plt.show()
sqaures.shape

In [ ]:
def image_to_squares(img,heights,widths):
  squares = []
  for i in range(0,8):
    for j in range(0,8):
      squares.append(img[i*heights//8:i*heights//8+heights//8,j*widths//8:j*widths//8+widths//8])
  return np.array(squares)

In [ ]:
sqaures= image_to_squares(preprocessed_imgs[0],240,240)
fig = plt.figure(figsize=(10, 10))
columns = 8
rows = 8
for i, img in zip(range(1, columns*rows +1),sqaures):
    fig.add_subplot(rows, columns, i)
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.tight_layout()
plt.show()
sqaures.shape

In [ ]:
def preprocess_image(img_path):
    height =240
    width =240
    img = cv2.imread(img_path, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.resize(img, (width, height))
    gray_image =(gray_image - np.min(gray_image)) / (np.max(gray_image) - np.min(gray_image))

    squares = image_to_squares(gray_image,height,width)
    return squares

In [ ]:
sqaures=preprocess_image(train[444])

fig = plt.figure(figsize=(10, 10))
columns = 8
rows = 8
for i, img in zip(range(1, columns*rows +1),sqaures):
    fig.add_subplot(rows, columns, i)
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.tight_layout()
plt.show()
sqaures.shape

In [ ]:
def onehot_from_fen(fen):
    eye = np.eye(13)
    output = np.empty((0, 13))
    fen = re.sub('[-]', '', fen)

    for char in fen:
        if char in '12345678':
              output = np.append(output, np.tile(eye[12], (int(char), 1)), axis=0)
        else:
              if char in piece_symbols:
                 idx = piece_symbols.index(char)
                 output = np.append(output, eye[idx].reshape((1, 13)), axis=0)
              else:
                pass

    return output


def fen_from_onehot(one_hot):
    output = ''
    for j in range(8):
        for i in range(8):
            if(one_hot[j][i] == 12):
                output += ' '
            else:
                output += piece_symbols[one_hot[j][i]]
        if(j != 7):
            output += '-'

    for i in range(8, 0, -1):
        output = output.replace(' ' * i, str(i))

    return output

In [ ]:
def train_gen(features):
    for i, img in enumerate(features):
        y = onehot_from_fen(get_image_FEN_label(img))
        x = preprocess_image(img)
        yield x, y

def pred_gen(features):
    for i, img in enumerate(features):
        y = onehot_from_fen(get_image_FEN_label(img))
        x = preprocess_image(img)
        yield x, y

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

# Add a convolutional layer
model = Sequential()
model.add(Convolution2D(32, (3, 3),activation='relu', input_shape=(30, 30, 3)))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Convolution2D(16, (5, 5),activation='relu'))
model.add(Flatten())
model.add(Dropout(0.35))
model.add(Dense(13, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:


hist = model.fit_generator(train_gen(train), steps_per_epoch=250, epochs=30,validation_data=pred_gen(test), validation_steps=30)


In [ ]:
model.save("content/datas/chess_model.h5")

In [ ]:
fig = plt.figure(figsize=(15, 5))
plt.plot(hist.history['val_loss'], label='(test data)')
plt.plot(hist.history['loss'], label='(train data)')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.legend(loc="upper right")
plt.title('Validation Loss')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 5))

plt.plot(hist.history['val_accuracy'], label='(test data)')
plt.plot(hist.history['accuracy'], label='(train data)')

plt.ylabel('value')
plt.xlabel('No. epoch')
plt.legend(loc="lower right")
plt.title('Validation Accuracy')
plt.show()

In [ ]:
res = (
  model.predict_generator(pred_gen(test), steps=test_size)
  .argmax(axis=1)
  .reshape(-1, 8, 8)
)



In [ ]:
pred_fens = np.array([fen_from_onehot(one_hot) for one_hot in res])
test_fens = np.array([get_image_FEN_label(fn) for fn in test])

final_accuracy = 100*(pred_fens == test_fens).astype(float).mean()

print("Testing Accuracy: {:1.2f}%".format(final_accuracy))

In [ ]:
print("\nConfusion Matrix:\n------------------------")
confusion_matrix(test_fens, pred_fens)

In [ ]:
model.save("content/datas/chess_model.h5")

In [ ]:
import matplotlib.image as mpimg
def display_with_predicted_fen(image):
    plt.figure(figsize=(5,5))
    pred = model.predict(preprocess_image(image)).argmax(axis=1).reshape(-1, 8, 8)
    fen = fen_from_onehot(pred[0])
    imgplot = plt.imshow(mpimg.imread(image))
    plt.axis('off')
    plt.title(fen)
    plt.show()
    return fen


In [ ]:
model.save("content/datas/chess_modelll.h5")

In [ ]:
image_path = "/content/testing_img"


predicted_fen_score=display_with_predicted_fen(image_path)
print("predicted FEN :",predicted_fen_score)
